
# 00-02 Lab · LangGraph Fundamentals 실습

프리 세션에서 살펴본 `interrupt` + `resume` 흐름을 직접 구현해보는 미션입니다. 아래 TODO 셀을 순서대로 완성하면서 **StateGraph 구성 → 분기 제어 → 재개 로직**을 체득하세요.



## 🎯 러닝 체크리스트
- `TypedDict`로 상태 스키마를 명확히 선언할 수 있다.
- `StateGraph`에 노드를 추가하고 `Command(goto=…)`로 분기 제어를 구현한다.
- `interrupt()`로 중단된 워크플로우를 `Command(resume=…)`로 재개하는 코드를 작성한다.

> ⚠️ 참고: 가이드 노트북(`00-01-langgraph-fundamentals-guide.ipynb`)을 먼저 실행해 필요한 라이브러리를 설치하고 개념을 복습한 뒤 실습을 진행하세요.


In [ ]:

# (선택) 의존성 설치 – 가이드 노트북에서 이미 실행했다면 건너뛰세요.
# !pip install -q langgraph langchain langchain-teddynote
# !pip install -q grandalf matplotlib networkx pyppeteer
# !pip install -q git+https://github.com/lgai-exaone/transformers@add-exaone4
# !pip install -q torch accelerate


In [ ]:

from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage

# 실습을 단순화하기 위해 LLM 호출 대신 문자열 포맷팅을 사용합니다.



## 1. 상태 스키마 설계
고객 문의를 받아 초안 메일을 만들고, 필요 시 인간 검토자가 피드백을 남긴 뒤 최종 메일을 확정하는 플로우를 구성하세요.
- `EmailState`에는 최소한 아래 키가 포함되어야 합니다.
  - `customer_message`: 원문 문의
  - `draft`: AI 초안 텍스트
  - `reviewer_feedback`: 검토자 피드백(없을 수도 있음)
  - `final_response`: 최종 발송 메일
  - 그 외 필요한 필드를 자유롭게 추가하세요.


In [ ]:

# TODO: EmailState TypedDict와 초기 상태 헬퍼를 정의하세요.
# 힌트: 프리 노트북에서 사용한 구조를 참고하되, 실습 요구사항에 맞게 필드를 정리합니다.

raise NotImplementedError("EmailState를 정의하고 초기 상태를 반환하는 함수를 구현하세요.")



## 2. 노드 로직 구현
세 가지 노드를 작성합니다.
1. `draft_email_node(state)` – 고객 문의를 바탕으로 초안을 작성하고 다음 단계로 이동 여부를 판단합니다.
2. `human_review_node(state)` – 검토자 피드백을 상태에 기록합니다.
3. `finalize_node(state)` – 초안과 피드백을 합쳐 최종 응답을 완성합니다.

조건 분기 규칙 예시:
- 고객 문의에 "긴급"이라는 단어가 포함되어 있거나, 초안 길이가 200자를 넘으면 인간 검토로 보내세요.
- 그 외에는 바로 `finalize_node`로 이동합니다.

> 필요하다면 추가 유틸 함수를 자유롭게 만들어 사용해도 됩니다.


In [ ]:

# TODO: draft_email_node / human_review_node / finalize_node를 구현하세요.
# 노드가 반환해야 하는 형식과 Command 사용법은 가이드 노트북을 참고하세요.

raise NotImplementedError("노드 로직을 채워 넣으세요.")



## 3. 그래프 빌드 및 실행 유틸
- `build_email_workflow()`에서 `StateGraph`를 구성하고 `MemorySaver` 체크포인터를 붙이세요.
- `run_email_flow(customer_message, reviewer_feedback)` 함수는 초기 상태를 만든 뒤 그래프를 실행하고 `final_response`를 반환해야 합니다.
- 인간 검토가 필요한 경우 `Command(resume=reviewer_feedback)`를 사용해 두 번째 단계에서 워크플로우를 재개하세요.


In [ ]:

# TODO: 그래프를 구성하고 실행 유틸 함수를 작성하세요.

raise NotImplementedError("build_email_workflow와 run_email_flow를 구현하세요.")



## 4. 빠른 자체 점검
다음 셀을 실행했을 때 모든 assert가 통과되면 기본 요구사항을 충족한 것입니다. 실패한다면 위 셀을 다시 점검하세요.


In [ ]:

# ✅ 간단한 테스트 (필요 시 자유롭게 수정/확장하세요.)

try:
    workflow = build_email_workflow()
    assert hasattr(workflow, "compile"), "StateGraph 인스턴스를 반환해야 합니다."
    sample = run_email_flow("배송 지연으로 환불이 필요해요", "환불 정책을 명확히 안내하고 사과문을 강화해주세요")
    assert "환불" in sample, "최종 응답에 고객 문의의 핵심 단어가 포함되어야 합니다."
    print("🎉 기본 테스트 통과! 추가 시나리오를 직접 만들어 검증해보세요.")
except NotImplementedError:
    print("⚠️ 아직 TODO가 남았습니다. 위 셀을 완성하세요.")



## 다음 단계 아이디어
- 검토자 피드백을 여러 번 받을 수 있도록 `thread_id` 기반 다중 재개 로직을 추가해보세요.
- 고객 유형(예: VIP, 신규, 일반)에 따라 라우팅 규칙을 세분화해보세요.
- `visualize_graph()`로 그래프를 출력하고 조건 분기 경로를 설명할 수 있는지 확인하세요.
